In [1]:
import csv
import numpy as np
import sys
import os
from osgeo import gdal
current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir,'..','src'))
from util import *

sys.path.append(os.path.join(current_dir,'..','params'))
from load_params import *



In [2]:

#obtention du tableau RVB à partir des 3 images satellites B2,B3,B4 
chemin = os.path.abspath("../image/image_sondage")
#chemin = os.path.abspath("../image/image_equation")

#Pensez bien à changer le nom de l'image
dossier = os.path.join(chemin,"NouvelleCaledonie/NouvelleCaledonie_10m")
img = "2024-11-18-0000_2024-11-18-2359_Sentinel-2_L2A"
#image 
#dossier = os.path.join(chemin,"Carrizo_Plain_California)")
#img = "2017-03-28-00:00_2017-03-28-23:59_Sentinel-2_L1C"


def RVB(root_srcpath, path_name, prod_name):
    bnames = ['B04','B03','B02']
    
    # Obtenir le chemin en entier
    srcpath = os.path.join(root_srcpath,path_name)
    srcfile = os.path.join(srcpath,prod_name+'_'+bnames[0]+'_Raw.tiff')
    src = gdal.Open(srcfile)
    
    # Obtenir les dimensions de l'image
    nx = src.RasterXSize
    ny = src.RasterYSize
    src = None
    
    # Préparer la matrice de sortie
    out_matrix = np.empty((nx * ny, 3), dtype=float)
    
    # Charger les bandes et remplir la matrice
    for i, bname in enumerate(bnames):
        srcfile = os.path.join(srcpath, prod_name + '_' + bname + '_Raw.tiff')
        src = gdal.Open(srcfile)
                
        # Lire la bande entière 
        arr = src.GetRasterBand(1).ReadAsArray().astype(float) 
        src = None
        
        out_matrix[:, i] = arr.flatten()
        
    return out_matrix, nx, ny
RVB, nx, ny = RVB(chemin, dossier, img)

RVB = np.clip(RVB, 0,1)
print(RVB.shape)
print(np.max(RVB),np.min(RVB))
#print(RVB)

(540260, 3)
0.7856000065803528 0.017500000074505806


In [3]:

#convertion de RVB en XYZ
T_8_new = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8_new
T_8 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq8
T_9 = matrix_s2l2ab4b3b2_to_xyzd65_sovdat2019_eq9

def RVB_to_XYZ(T, A):
    B = A @ T
    return B

XYZ = RVB_to_XYZ(T_8_new, RVB)
print(np.shape(XYZ))


(540260, 3)


In [4]:

# XYZ to sRVB

#standart Martix define by CIE
M = np.array([[3.240625,-1.537208,-0.498629],
             [-0.968931,1.875756,0.041518],
             [0.055710, -0.204021,1.056996]])

sRVB = M@XYZ.T
#print(sRVB)
print(np.max(sRVB),np.min(sRVB))
sRVB = np.clip(sRVB, 0, 1)
print(sRVB.shape)

RVB_16bit = np.clip(sRVB*65535, 0, 65535)
#print(sRVB)
#print(RVB_16bit)



0.855546057839677 0.02081000062437088
(3, 540260)


In [5]:
"""
#RVB équation T8/T9 en sRVB 8bit et application gamma 2.4
R = sRVB[0]
V = sRVB[1]
B = sRVB[2]

def phi(R):
    for k in range(len(R)):
        if R[k] <= 0.0031308:
            R[k] = R[k]*255*12.92
        else:
            R[k] = 255*(1.055*R[k]**(1/2.4) - 0.055)
    return R

R_8bit = phi(R)
V_8bit = phi(V)
B_8bit = phi(B)
sRVB_8bit = np.array([R_8bit,V_8bit,B_8bit]).T
sRVB_8bit = np.clip(sRVB_8bit, 0, 255).astype(np.uint8)
print(sRVB_8bit)
#print(np.max(sRVB))
#print(np.shape(sRVB_8bit))
"""

'\n#RVB équation T8/T9 en sRVB 8bit et application gamma 2.4\nR = sRVB[0]\nV = sRVB[1]\nB = sRVB[2]\n\ndef phi(R):\n    for k in range(len(R)):\n        if R[k] <= 0.0031308:\n            R[k] = R[k]*255*12.92\n        else:\n            R[k] = 255*(1.055*R[k]**(1/2.4) - 0.055)\n    return R\n\nR_8bit = phi(R)\nV_8bit = phi(V)\nB_8bit = phi(B)\nsRVB_8bit = np.array([R_8bit,V_8bit,B_8bit]).T\nsRVB_8bit = np.clip(sRVB_8bit, 0, 255).astype(np.uint8)\nprint(sRVB_8bit)\n#print(np.max(sRVB))\n#print(np.shape(sRVB_8bit))\n'

In [6]:
"""
#Fonctionne pas pour Luminence HDR

#transformation en tiff
def matrix_to_image(out_matrix, nx, ny, output_filename):
    # Reshape the matrix to its original 2D form

    b4 = out_matrix[:, 0].reshape((ny, nx))  # B04
    b3 = out_matrix[:, 1].reshape((ny, nx))  # B03
    b2 = out_matrix[:, 2].reshape((ny, nx))  # B02

    # Stack the three bands into an RGB image
    rgb_image = np.dstack((b4, b3, b2))

    # Save the image (using GDAL)
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_filename, nx, ny, 3, gdal.GDT_UInt16)
    
    #Pour les images 8bits
    #out_ds = driver.Create(output_filename, nx, ny, 3, gdal.GDT_Byte)
    
    out_ds.GetRasterBand(1).WriteArray(b4)
    out_ds.GetRasterBand(2).WriteArray(b3)
    out_ds.GetRasterBand(3).WriteArray(b2)
    
    out_ds.FlushCache()
    out_ds = None  # Close file
    

output_filename = "../image/image_sondage/Alpes/T8/Alpes_10m_16bits.tiff"
matrix_to_image(RVB_16bit.T, nx, ny, output_filename)
"""


'\n#Fonctionne pas pour Luminence HDR\n\n#transformation en tiff\ndef matrix_to_image(out_matrix, nx, ny, output_filename):\n    # Reshape the matrix to its original 2D form\n\n    b4 = out_matrix[:, 0].reshape((ny, nx))  # B04\n    b3 = out_matrix[:, 1].reshape((ny, nx))  # B03\n    b2 = out_matrix[:, 2].reshape((ny, nx))  # B02\n\n    # Stack the three bands into an RGB image\n    rgb_image = np.dstack((b4, b3, b2))\n\n    # Save the image (using GDAL)\n    driver = gdal.GetDriverByName(\'GTiff\')\n    out_ds = driver.Create(output_filename, nx, ny, 3, gdal.GDT_UInt16)\n    \n    #Pour les images 8bits\n    #out_ds = driver.Create(output_filename, nx, ny, 3, gdal.GDT_Byte)\n    \n    out_ds.GetRasterBand(1).WriteArray(b4)\n    out_ds.GetRasterBand(2).WriteArray(b3)\n    out_ds.GetRasterBand(3).WriteArray(b2)\n    \n    out_ds.FlushCache()\n    out_ds = None  # Close file\n    \n\noutput_filename = "../image/image_sondage/Alpes/T8/Alpes_10m_16bits.tiff"\nmatrix_to_image(RVB_16bit.T, n

In [7]:
"""
Fonctionne que pour les 8bits
#image RVB direct
import imageio.v2 as imageio

sRVB_direct = np.clip(RVB*255, 0, 255)
print(sRVB_direct.shape)


output_filename = "../image/image_equation/Amposta_Spain/Amposta_Spain_direct.tiff"
matrix_to_image(sRVB_direct_img, nx, ny, output_filename)
"""

'\nFonctionne que pour les 8bits\n#image RVB direct\nimport imageio.v2 as imageio\n\nsRVB_direct = np.clip(RVB*255, 0, 255)\nprint(sRVB_direct.shape)\n\n\noutput_filename = "../image/image_equation/Amposta_Spain/Amposta_Spain_direct.tiff"\nmatrix_to_image(sRVB_direct_img, nx, ny, output_filename)\n'

In [8]:

#Fonctionne pour les 16bits

import imageio.v2 as imageio

#Matrice 16bits
RVB_img_ = RVB_16bit.T.astype(np.uint16) #Attention bien transposé
RVB_img = RVB_img_.reshape(ny,nx,3)


# Transformer en un format supporté par imageio (par exemple PNG)

#output_tiff_filename = "../image/image_sondage/Arabie_Saoudite/T8/Arabie_Saoudite_20m_16bits.tiff"
output_filename = "../image/image_sondage/NouvelleCaledonie/T8/NouvelleCaledonie_10m_16bits.tiff"

# Enregistrer l'image
imageio.imwrite(output_filename, RVB_img,  format='TIFF')



In [9]:
#Fonctionne que pour les 16 bits
import imageio.v2 as imageio

#Matrice RVB 16bits
RVB_ = np.clip(RVB*65535, 0,65535).astype(np.uint16)
sRVB_img = RVB_.reshape(ny,nx,3)
print(sRVB_img.shape)
print(RVB.shape)


# Transformer en un format supporté par imageio (par exemple PNG)

#output_tiff_filename = "../image/image_sondage/Arabie_Saoudite/T8/Arabie_Saoudite_20m_16bits.tiff"
output_tiff_filename = "../image/image_sondage/NouvelleCaledonie/NouvelleCaledonie_16bits.tiff"

# Enregistrer l'image
imageio.imwrite(output_tiff_filename, sRVB_img,  format='TIFF')


(595, 908, 3)
(540260, 3)
